In [1]:
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

import subprocess
from matplotlib.animation import FuncAnimation
import matplotlib.patches as patches
from matplotlib.colors import ListedColormap
from scipy.io import loadmat
import natsort, pathlib
import os
import glob

from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [2]:
colormap_data = loadmat('cold2hot.mat')
cold2hot = ListedColormap(colormap_data['colormap'])
hot2cold = ListedColormap(colormap_data['colormap'][::-1])

In [3]:
path = os.path.abspath('../0_MOOSE_Input_Files/Heat_Sources/')

In [4]:
model_path_BHS = path+'/Bessel/TDB/exodus/Au-Au.e'
model_path_GHS = path+'/Gaussian/TDB/exodus/Au-Au.e'
model_path_FTHS = path+'/FlatTop/TDB/exodus/Au-Au.e'
model_path_RHS = path+'/Ring/TDB/exodus/Au-Au.e'

In [23]:
model = netCDF4.Dataset(model_path_RHS)

In [24]:
names = model.variables["name_nod_var"]
names.set_auto_mask(False)
param_in_exodus = [b"".join(c).decode("latin1") for c in names[:]]
print(param_in_exodus)

['eta1', 'p', 'temp', 'vel_x', 'vel_y']


In [25]:
parameters = ['vals_nod_var1', 'vals_nod_var3', 'vals_nod_var4', 'vals_nod_var5',]

In [26]:
LIQ, temp, velX, velY = [], [], [], []

for i in range(4):
    param = model.variables[parameters[i]][:]

    X_all = model.variables['coordx'][:]
    Y_all = model.variables['coordy'][:]
    
    points = np.vstack([Y_all,X_all,param]).T
    
    column_names = ['X', 'Y'] + list(range(param.shape[0]))
    df = pd.DataFrame(points, columns=column_names)
    df = df.sort_values(by = ['X', 'Y'], ascending = [True, True], ignore_index=True)
    df_param = df.iloc[:, 2:]
    
    x_dim  = model.dimensions['num_nod_ns1'].size
    y_dim =  model.dimensions['num_nod_ns2'].size
    time_steps = model.dimensions['time_step'].size
    array_ = df_param.to_numpy().flatten().reshape(y_dim, x_dim, time_steps)
    array = np.flip(array_, axis=0)
    if i == 0:
        LIQ.append(array)
    elif i == 1:
        temp.append(array)
    elif i == 2:
        velX.append(array)
    else:
        velY.append(array)

# time = np.array(model.variables['time_whole'][:]).reshape((-1, ))
# LIQ, temp, = np.clip(LIQ[0][:,:,:], 0, 1), temp[0][:,:,:]
# FCC = 1- LIQ

In [27]:
## BHS 66;477 | GHS 52;455 | FTHS 62;469 | RHS 17;419 
source_type = "RHS"

if source_type == "BHS":
    t_ST, time_steps = 66, 477
elif source_type == "GHS":
    t_ST, time_steps = 52, 455
elif source_type == "FTHS":
    t_ST, time_steps = 62, 469
else:
    t_ST, time_steps = 17, 419

dt = time_steps - t_ST
time = np.array(model.variables['time_whole'][:]).reshape((-1, ))[np.r_[t_ST:time_steps]]
LIQ, temp, velX, velY = np.where(LIQ[0] < 0.5, 0, 1)[:, :, np.r_[t_ST:time_steps]], temp[0][:, :, np.r_[t_ST:time_steps]], velX[0][:, :, np.r_[t_ST:time_steps]], velY[0][:, :, np.r_[t_ST:time_steps]]
vel = (velX**2 + velY**2)**0.5

## BHS 2;max | GHS 2;max | FTHS 2;max | RHS 2;max 
vel = np.round(np.where(vel < 2, 0, vel), decimals=2)  # Setting velocity in SOLID (FCC) Region to 0 and rounding off the decimal to 2 places

X_, y_, V_, = np.moveaxis(temp, -1, 0), np.moveaxis(LIQ, -1, 0), np.moveaxis(vel, -1, 0)

dt
# temperature = np.where(temp < 1337, 0, 1)
# X_, y_ = np.moveaxis(temperature, -1, 0), np.moveaxis(LIQ, -1, 0)

402

In [ ]:
411+403+407+402

In [10]:
X, y, V, TIME = X_, y_, V_, time.copy() # Run this only for the first time

In [28]:
X, y, V, TIME = np.concatenate((X, X_), axis=0), np.concatenate((y, y_), axis=0), np.concatenate((V, V_), axis=0), np.concatenate((TIME, time), axis=0)
X.shape, y.shape, V.shape, TIME.shape

((1623, 201, 401), (1623, 201, 401), (1623, 201, 401), (1623,))

In [ ]:
t_step = 150
laser_speed = 30
laser_pos = (125 + time[t_step]*laser_speed)* 401/1000  # Laser actual position in true dimension

fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(12,12), frameon=True)
cmap = plt.get_cmap('RdYlGn_r')
cmap.set_under('white', alpha=0)
hmap1 = ax1.imshow(y_[0], cmap=cmap, vmin=0.5, vmax=1.0, aspect=0.5,  interpolation='quadric')
cmap = plt.get_cmap('Wistia')
cmap.set_under('white', alpha=0) 
hmap2 = ax1.imshow(1-y_[0], cmap=cmap, vmin=0.5, vmax=1.5, aspect=0.5, interpolation='quadric')
ax1.arrow(50.125,-75, 0, 71,  width = 8.5, color='red', length_includes_head=True, clip_on=False)
ax1.set_title(f't = {time[0]:.2f}s', pad=10, loc='right', fontsize=25, weight='bold')

cmap = plt.get_cmap('RdYlGn_r')
cmap.set_under('white', alpha=0)
hmap3 = ax2.imshow(y_[t_step], cmap=cmap, vmin=0.5, vmax=1.0, aspect=0.5,  interpolation='quadric')
cmap = plt.get_cmap('Wistia')
cmap.set_under('white', alpha=0) 
hmap4 = ax2.imshow(1-y_[t_step], cmap=cmap, vmin=0.5, vmax=1.5, aspect=0.5, interpolation='quadric')
ax2.arrow(laser_pos,-75, 0, 71,  width = 8.5, color='red', length_includes_head=True, clip_on=False)
ax2.set_title(f't = {time[t_step]:.2f}s', pad=10, loc='right', fontsize=25, weight='bold')


cmap = plt.get_cmap('gist_ncar_r')
cmap.set_under('white', alpha=0)
hmap5 = ax3.imshow(V_[t_step], cmap=cmap, vmin=2, aspect=0.5,  interpolation='quadric')
ax3.arrow(laser_pos,-75, 0, 71,  width = 8.5, color='red', length_includes_head=True, clip_on=False)
ax3.set_title(f't = {time[t_step]:.2f}s', pad=10, loc='right', fontsize=25, weight='bold')


for ax in [ax1, ax2, ax3]: ax.set_ylim(201, -1); ax.set_xlim(-1, 401); ax.tick_params(axis='both', labelcolor='black', labelsize=65, bottom=False, top=False, left=False, right=False, labelbottom=False, labelleft=False); ax.spines[:].set_linewidth(4)
for label in ax2.clabel(ax2.contour(X_[t_step], levels=[450, 700, 1337], colors='deepskyblue', linewidths=5, linestyles='dashdot'), inline=True, fontsize=25, colors='k', fmt='%1.0f K'): label.set_fontweight('bold')
for label in ax3.clabel(ax3.contour(y_[t_step], levels=[0.99], colors='red', linewidths=5, linestyles='-'), inline=True,): label.set_fontweight('bold')

In [29]:
##########################################
# Saving as Numpy Files
path = os.path.abspath('../1_Simulation_Results/ML_numpy_files/')
np.save(path+'/Temperature.npy', X)
np.save(path+'/Phase.npy', y)
np.save(path+'/Velocity.npy', V)
np.save(path+'/Time.npy', TIME)

# Splitting Train Test Validation Dataset

In [ ]:
path = os.path.abspath('data/')
X, y = np.load(path+'/X.npy'), np.load(path+'/y.npy')

In [ ]:
## Train Validation Test Split
# Reshape X and y to match PyTorch's Conv2d input format: (batch_size, channels, width, height)
X_reshaped = X[:, np.newaxis, :, :] 
y_reshaped = y[:, np.newaxis, :, :] 

# Split data into training, testing, validation sets
X_, X_test, y_, y_test = train_test_split(X_reshaped, y_reshaped, test_size=0.1, random_state=69)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.111, random_state=69)  #test_size=0.109


In [ ]:
np.save('data/X_test.npy', X_test)
np.save('data/y_test.npy', y_test)

## OLD

In [ ]:
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

import subprocess
from matplotlib.animation import FuncAnimation
import matplotlib.patches as patches
from matplotlib.colors import ListedColormap
from scipy.io import loadmat
import natsort, pathlib
import os
import glob

In [ ]:
colormap_data = loadmat('cold2hot.mat')
cold2hot = ListedColormap(colormap_data['colormap'])
hot2cold = ListedColormap(colormap_data['colormap'][::-1])

In [ ]:
path = os.path.abspath('../0_MOOSE_Input_Files/Heat_Sources/ML_Data/Bessel/TDB/exodus/')

In [ ]:
model_path = path+'/Au-Au.e'
model = netCDF4.Dataset(model_path)

In [ ]:
model.dimensions

In [ ]:
model.variables.keys()

In [ ]:
names = model.variables["name_nod_var"]
names.set_auto_mask(False)
param_in_exodus = [b"".join(c).decode("latin1") for c in names[:]]
print(param_in_exodus)

In [ ]:
parameters = ['vals_nod_var1', 'vals_nod_var3',]

In [ ]:
area_df1 = pd.read_csv(path+'/Au-Au.csv',  encoding='utf-8')
real_time = np.array(area_df1['time'])
area_df1.head()

In [ ]:
LIQ, temp = [], []

for i in range(2):
    param = model.variables[parameters[i]][:]

    X_all = model.variables['coordx'][:]
    Y_all = model.variables['coordy'][:]
    
    points = np.vstack([Y_all,X_all,param]).T
    
    column_names = ['X', 'Y'] + list(range(param.shape[0]))
    df = pd.DataFrame(points, columns=column_names)
    df = df.sort_values(by = ['X', 'Y'], ascending = [True, True], ignore_index=True)
    df_param = df.iloc[:, 2:]
    
    x_dim  = model.dimensions['num_nod_ns1'].size
    y_dim =  model.dimensions['num_nod_ns2'].size
    time_steps = model.dimensions['time_step'].size
    array_ = df_param.to_numpy().flatten().reshape(y_dim, x_dim, time_steps)
    array = np.flip(array_, axis=0)
    if i == 0:
        LIQ.append(array)
    else:
        temp.append(array)

# time = np.array(model.variables['time_whole'][:]).reshape((-1, ))
# LIQ, temp, = np.clip(LIQ[0][:,:,:], 0, 1), temp[0][:,:,:]
# FCC = 1- LIQ

In [ ]:
t_ST = 61
time_steps = 467
dt = time_steps - t_ST
time = np.array(model.variables['time_whole'][:]).reshape((-1, ))[np.r_[t_ST:time_steps]]
LIQ, temp = np.where(LIQ[0] < 0.5, 0, 1)[:, :, np.r_[t_ST:time_steps]], temp[0][:, :, np.r_[t_ST:time_steps]]

temperature = np.where(temp < 1337, 0, 1)
LIQ, temperature = np.moveaxis(LIQ, -1, 0), np.moveaxis(temperature, -1, 0)
# LIQ, temperature = LIQ.reshape(dt, 201 * 401), temperature.reshape(dt, 201 * 401)

In [ ]:
##########################################
# Saving as Numpy Files
path = os.path.abspath('../1_Simulation_Results/ML_numpy_files/')
np.save(path+'/X.npy', temperature)
np.save(path+'/y.npy', LIQ)
np.save(path+'/time.npy', time)

In [ ]:
LIQ, temperature = LIQ.reshape(dt, 201, 401), temperature.reshape(dt, 201, 401)

In [ ]:
t_step = -1
laser_speed = 30
laser_pos = (125 + time[t_step]*laser_speed)* 401/1000  # Laser actual position in true dimension

fig, (ax1, ax2) = plt.subplots(2,1, figsize=(12,8), frameon=True)
cmap = plt.get_cmap('RdYlGn_r')
cmap.set_under('white', alpha=0)
hmap1 = ax1.imshow(LIQ[0], cmap=cmap, vmin=0.5, vmax=1.0, aspect=0.5,  interpolation='quadric')
cmap = plt.get_cmap('Wistia')
cmap.set_under('white', alpha=0) 
hmap2 = ax1.imshow(1-LIQ[0], cmap=cmap, vmin=0.5, vmax=1.5, aspect=0.5, interpolation='quadric')
ax1.arrow(50.125,-75, 0, 71,  width = 8.5, color='red', length_includes_head=True, clip_on=False)
ax1.set_title(f't = {time[0]:.2f}s', pad=10, loc='right', fontsize=25, weight='bold')

cmap = plt.get_cmap('RdYlGn_r')
cmap.set_under('white', alpha=0)
hmap3 = ax2.imshow(LIQ[t_step], cmap=cmap, vmin=0.5, vmax=1.0, aspect=0.5,  interpolation='quadric')
cmap = plt.get_cmap('Wistia')
cmap.set_under('white', alpha=0) 
hmap4 = ax2.imshow(1-LIQ[t_step], cmap=cmap, vmin=0.5, vmax=1.5, aspect=0.5, interpolation='quadric')
ax2.arrow(laser_pos,-75, 0, 71,  width = 8.5, color='red', length_includes_head=True, clip_on=False)
ax2.set_title(f't = {time[t_step]:.2f}s', pad=10, loc='right', fontsize=25, weight='bold')

for ax in [ax1, ax2]: ax.set_ylim(201, -1); ax.set_xlim(-1, 401); ax.tick_params(axis='both', labelcolor='black', labelsize=65, bottom=False, top=False, left=False, right=False, labelbottom=False, labelleft=False); ax.spines[:].set_linewidth(4)
for label in ax2.clabel(ax2.contour(temp[:, :, t_step], levels=[450, 700, 1337], colors='deepskyblue', linewidths=5, linestyles='dashdot'), inline=True, fontsize=25, colors='k', fmt='%1.0f K'): label.set_fontweight('bold')